In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from konlpy.tag import Mecab
import nltk
from collections import Counter
import itertools
import pickle

In [3]:
# target_day = (datetime.datetime.today()-timedelta(months=12)).strftime('%Y-%m-%d')
target_day = (datetime.datetime.today()-relativedelta(months = 3)).strftime('%Y-%m-%d')
target_day

'2023-03-15'

In [4]:
target_day = '2023-03-31'

In [5]:
def get_recruit(recruit_list):
    page_cnt = 1
    flag = 1
    while(flag == 1):
        if page_cnt == 1:
            url = "http://kofia.or.kr/brd/m_96/list.do"
            link_url = "http://www.kofia.or.kr/brd/m_96/"
            response = requests.get(url)
            soup = BeautifulSoup(response.content, "html.parser")
            trs = soup.find_all('tr')
            for tr in trs[1:]:
                post_dt = tr.select('td.num')[1].get_text()
                if post_dt > target_day:
                    td = tr.select("td.left")[0]
        #             td = tr.select("td.left.new")[0]
                    a = td.select("a")[0]
                    href = a.get("href")
                    link = link_url + href
                    recruit_list.append(link)
                else:
                    flag = 0
            page_cnt += 1
            page_url = "?page="+str(page_cnt)+"&amp;srchFr=&amp;srchTo=&amp;srchWord=&amp;srchTp=&amp;multi_itm_seq=0&amp;itm_seq_1=0&amp;itm_seq_2=0&amp;company_cd=&amp;company_nm="
        else:
            url = "http://kofia.or.kr/brd/m_96/list.do" + page_url
            link_url = "http://www.kofia.or.kr/brd/m_96/"
            response = requests.get(url)
            soup = BeautifulSoup(response.content, "html.parser")
            trs = soup.find_all('tr')
            for tr in trs[1:]:
                post_dt = tr.select('td.num')[1].get_text()
                if post_dt > target_day:
                    td = tr.select("td.left")[0]
                    a = td.select("a")[0]
                    href = a.get("href")
                    link = link_url + href
                    recruit_list.append(link)
                else:
                    flag = 0  
            page_cnt += 1
            page_url = "?page="+str(page_cnt)+"&amp;srchFr=&amp;srchTo=&amp;srchWord=&amp;srchTp=&amp;multi_itm_seq=0&amp;itm_seq_1=0&amp;itm_seq_2=0&amp;company_cd=&amp;company_nm="
            
    return recruit_list

In [6]:
recruit_list = []
recruit_list = get_recruit(recruit_list)

In [7]:
rows = []
for rec in recruit_list:
    response = requests.get(rec)
    soup = BeautifulSoup(response.content, "html.parser")
    ths = soup.tbody.find_all('th')
    for th in ths:
        if th.get_text()=='회원사명':
            com_tr = th.parent
            break
    com = com_tr.find('td').get_text()
    for th in ths:
        if th.get_text()=='등록일':
            dt_tr = th.parent
            break
    dt = dt_tr.find('td').get_text()
    title = soup.title.get_text()
    contents = soup.select('td.first.lastline')[0].get_text().strip('\n').strip('\t')
    rows.append({'등록일':dt, '회사명':com, '제목':title, '내용':contents})
df = pd.DataFrame(rows)
df.head()

,등록일,회사명,제목,내용
0,2023-06-14 18:04:02,한양증권,[한양증권] T&S부(결제업무부) 경력직원 채용 상세보기|채용안내 | 금융투자협회,[한양증권] T&S부(결제업무부) 경력직원 채용1. 모집부문1) 근무부서 : T&S...
1,2023-06-14 17:53:28,한양증권,[한양증권] 재무회계부 경력직원 채용 상세보기|채용안내 | 금융투자협회,[한양증권] 재무회계부 경력직원 채용1. 모집부문1) 근무부서 : 재무회계부2) 주...
2,2023-06-14 17:50:43,한국포스증권,[한국포스증권(주)] 리스크 관리 담당 채용공고(~07.09.) 상세보기|채용안내 ...,Ⅰ. 회사소개 - 2013년 국내 40여개 자산운용사와 증권 유관 기관들이 공동...
3,2023-06-14 17:49:35,한양증권,[한양증권] AI운용부문(주식운용) 업무직 채용 상세보기|채용안내 | 금융투자협회,[한양증권] AI운용부문(주식운용) 업무직 채용1. 모집부문1) 근무부서 : AI운...
4,2023-06-14 17:15:13,신한자산신탁,"[신한자산신탁] 경력직원 수시채용 (사내변호사, 신탁회계) 상세보기|채용안내 | 금...",신한자산신탁에서 꿈을 펼칠 열정 넘치는 직원들을 모집합니다.□ 모집분야 및 지원자격...


In [8]:
df.shape

(752, 4)

In [9]:
with open('./data/recruit_df_0615.pkl', 'wb') as f:
    pickle.dump(df, f)

In [10]:
with open('./data/recruit_df_0615.pkl', 'rb') as f:
    df = pickle.load(f)
df.shape

(752, 4)

#### 사용자 정의 사전 구축

In [6]:
# 회사명(회원사명) 전체 긁어와서 distinct로 사용자 정의 사전에 insert
# user_dic = []
# for index, row in df.iterrows():
#     user_dic.append(row['회사명'])
# user_dic = list(set(user_dic))

#### Tokenizing

In [11]:
pos = ['NNG', 'NNP', 'NNB', 'SL']
m = Mecab('C:\Windows\mecab\mecab-ko-dic')

In [12]:
def tokenizer(text):
    tokens = m.pos(text)
    return [word for word, pos in filter(lambda x: (x[1] in pos), tokens)]

In [13]:
df['TOKENS'] = df['내용'].apply(tokenizer)
df.head()

,등록일,회사명,제목,내용,TOKENS
0,2023-04-21 09:32:01,유나이티드파트너스자산운용,[유나이티드파트너스자산운용] 투자팀 팀장급 채용 공고 상세보기|채용안내 | 금융투자협회,"■ 유나이티드파트너스 자산운용(주) 투자팀 팀장급 채용 공고""업계의 풍부한 경험을 ...","[유나이티, 드, 파트, 너스, 자산운용, 주, 투자, 팀, 팀장, 급, 채용, 공..."
1,2023-04-21 08:41:58,써밋자산운용,[써밋자산운용] IB부문 전문인력 / 경영관리 경력채용 상세보기|채용안내 | 금융투자협회,[써밋자산운용]에서 함께 도약할 유능한 인재를 모십니다.▣ 모집부문 및 상세내용1....,"[써밋자산운용, 도약, 유능, 인재, 모집, 부문, 상세, 내용, IB, 부문, 담..."
2,2023-04-21 08:14:04,DB금융투자,[DB금융투자] 해외대체상품 부문 경력사원 채용공고 상세보기|채용안내 | 금융투자협회,"◆ 담당업무- 해외 투자사 대상 상품 구매, 관리 및 PA 계약- 투자자 대상 해외...","[담당, 업무, 해외, 투자, 대상, 상품, 구매, 관리, PA, 계약, 투자자, ..."
3,2023-04-20 19:40:20,우리종합금융,[우리종합금융]DCM부 실무 및 영업(경력) 계약직 채용 상세보기|채용안내 | 금융...,"[모집부문]1. DCM부 경력직 (계장급) 0명[담당업무]1) 채권, CP, 단기사...","[모집, 부문, DCM, 부, 경력직, 계장, 급, 담당, 업무, 채권, CP, 단..."
4,2023-04-20 17:37:57,케이비자산운용,[KB자산운용] 인덱스퀀트본부 채용공고(~5/15) 상세보기|채용안내 | 금융투자협회,KB자산운용㈜ 인덱스퀀트본부에서 직원을 모집하오니 많은 관심과 지원 바랍니다. □ ...,"[KB, 자산운용, 인덱스, 퀀트, 본부, 직원, 모집, 관심, 지원, 모집, 부분..."


#### StopWords

In [14]:
def stopwords(tokens):
    stopwords = pd.read_csv('./data/stopwords.csv', encoding='utf-8-sig')
    stoplist = []
    for word in stopwords.word:
        stoplist.append(word)
    return list(set(tokens)-set(stoplist))

In [15]:
df['TOKENS_SW'] = df['TOKENS'].apply(stopwords)
df.head()

,등록일,회사명,제목,내용,TOKENS,TOKENS_SW
0,2023-04-21 09:32:01,유나이티드파트너스자산운용,[유나이티드파트너스자산운용] 투자팀 팀장급 채용 공고 상세보기|채용안내 | 금융투자협회,"■ 유나이티드파트너스 자산운용(주) 투자팀 팀장급 채용 공고""업계의 풍부한 경험을 ...","[유나이티, 드, 파트, 너스, 자산운용, 주, 투자, 팀, 팀장, 급, 채용, 공...","[시장, 경험, 드, 인재, 주선, 자산운용, 유나이티, 자격증, 팀장, 딜, 서울..."
1,2023-04-21 08:41:58,써밋자산운용,[써밋자산운용] IB부문 전문인력 / 경영관리 경력채용 상세보기|채용안내 | 금융투자협회,[써밋자산운용]에서 함께 도약할 유능한 인재를 모십니다.▣ 모집부문 및 상세내용1....,"[써밋자산운용, 도약, 유능, 인재, 모집, 부문, 상세, 내용, IB, 부문, 담...","[출구, 증명서, 자문, 세무, 인재, 주선, 도약, 선, 출신, 대리, 자격증, ..."
2,2023-04-21 08:14:04,DB금융투자,[DB금융투자] 해외대체상품 부문 경력사원 채용공고 상세보기|채용안내 | 금융투자협회,"◆ 담당업무- 해외 투자사 대상 상품 구매, 관리 및 PA 계약- 투자자 대상 해외...","[담당, 업무, 해외, 투자, 대상, 상품, 구매, 관리, PA, 계약, 투자자, ...","[조기, 실무, 증빙, 정규직, 판매, 영어, 전환, 여의도, 달성, 선, m, 국..."
3,2023-04-20 19:40:20,우리종합금융,[우리종합금융]DCM부 실무 및 영업(경력) 계약직 채용 상세보기|채용안내 | 금융...,"[모집부문]1. DCM부 경력직 (계장급) 0명[담당업무]1) 채권, CP, 단기사...","[모집, 부문, DCM, 부, 경력직, 계장, 급, 담당, 업무, 채권, CP, 단...","[DCM, 실무, 일반, 소공, 재무, 자금, 사채, 영업, 자료, 잡, 파악, 단..."
4,2023-04-20 17:37:57,케이비자산운용,[KB자산운용] 인덱스퀀트본부 채용공고(~5/15) 상세보기|채용안내 | 금융투자협회,KB자산운용㈜ 인덱스퀀트본부에서 직원을 모집하오니 많은 관심과 지원 바랍니다. □ ...,"[KB, 자산운용, 인덱스, 퀀트, 본부, 직원, 모집, 관심, 지원, 모집, 부분...","[증빙, 참고, 자산운용, 자료, 퀀트, 형식, 관심, 인덱스, 건강, KB]"
